In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
import tarfile
import os
import sys
import fcntl
import threading
import importlib.util

# Initialize Snowflake session
session = get_active_session()

# List of packages to be installed (upload azure-core and uamqp to your stage as well)
list_of_packages = [
    "azure-servicebus-7.12.2.tar.gz", 
    "uamqp-1.6.10.tar.gz"
]

# Fetch the package from the Snowflake stage
for pkg in list_of_packages:
    session.file.get(f"@PYTHON_PACKAGES/{pkg}", os.getcwd())

# File lock class for synchronizing access to /tmp during installation
class FileLock:
    def __enter__(self):
        self._lock = threading.Lock()
        self._lock.acquire()
        self._fd = open('/tmp/lockfile.LOCK', 'w+')
        fcntl.lockf(self._fd, fcntl.LOCK_EX)

    def __exit__(self, type, value, traceback):
        self._fd.close()
        self._lock.release()

# Function to extract the .tar.gz package and manually rebuild the module path
def install_tar_gz_package(pkg):
    extracted = '/tmp/python_pkg_dir'  # Directory where the package will be extracted
    tar_file_path = os.path.join(os.getcwd(), pkg)  # Path to the downloaded .tar.gz file

    # Ensure the extracted directory exists
    if not os.path.isdir(extracted):
        os.makedirs(extracted, exist_ok=True)

    # Extract the tar.gz file
    with tarfile.open(tar_file_path, "r:gz") as tar:
        tar.extractall(path=extracted)

    # Rebuild the module manually by directly adding the relevant folder to sys.modules
    package_name = pkg.replace(".tar.gz", "")
    package_dir = os.path.join(extracted, package_name)

    if os.path.isdir(package_dir):
        # Add the package directory to sys.path so Python can import it
        sys.path.append(package_dir)

# Extract and install the packages under a file lock to avoid concurrency issues
with FileLock():
    for pkg in list_of_packages:
        install_tar_gz_package(pkg)

# Try loading the azure module
try:
    importlib.import_module('azure.servicebus')
    st.write("Azure Service Bus Client Imported Successfully!")
except ImportError as e:
    st.write(f"Failed to import Azure Service Bus Client: {e}")


In [ ]:
import asyncio
from azure.servicebus import ServiceBusClient, ServiceBusMessage
from snowflake.snowpark.context import get_active_session

# Snowflake session initialization
session = get_active_session()

# Extract connection string for Service Bus from secret
api_secret_query = "SELECT python_workloads.profisee_mdm_prd.get_secret('cred2')"
connection_string = session.sql(api_secret_query).collect()[0][0]
topic_name = "account-source-record-topic"
subscription_name = "snowflake-subscription"
print(connection_string)

# Create a ServiceBusClient Client
servicebus_client = ServiceBusClient.from_connection_string(conn_str=connection_string, logging_enable=True)


# Function to append messages into Snowflake table
async def insert_into_snowflake(messages):
    # Create a DataFrame from the retrieved messages
    df = session.create_dataframe(messages, schema=["raw_data"])

    # Insert data into Snowflake
    df.write.mode("append").save_as_table("BI.PROFISEE.RAW_EIS_EGRESS")
    print(f"Appended {len(messages)} messages to Snowflake.")

# Asynchronous function to fetch and process messages in batches from Service Bus
async def async_peek_messages(receiver, batch_size):
    print(f"Peeking {batch_size} messages...")

    # Receive messages asynchronously
    messages = receiver.receive_messages(max_message_count=batch_size, max_wait_time=5)

    if messages:
        # Extract message bodies to prepare for Snowflake insertion
        message_bodies = [(str(message)) for message in messages]

        # Insert the batch into Snowflake
        await insert_into_snowflake(message_bodies)

        # Complete (lock) messages only if the insertion is successful
        for message in messages:
            receiver.complete_message(message)
            print(f"Message {message.message_id} completed and locked.")

    return len(messages)

# Function to initiate the asyncio event loop for peeking and processing messages in batches until no more messages are found
async def async_process_messages_continuously(batch_size=10, max_concurrent_batches=3):
    total_messages_processed = 0
    consecutive_empty_batches = 0  # Count for consecutive empty batches

    with servicebus_client:
        receiver = servicebus_client.get_subscription_receiver(
            topic_name=topic_name,
            subscription_name=subscription_name,
            max_wait_time=5  # Timeout for receiving messages
        )

        with receiver:
            while True:
                # Process messages in batches, peeking until no more messages
                semaphore = asyncio.Semaphore(max_concurrent_batches)
                task = asyncio.create_task(async_peek_messages(receiver, batch_size))

                # Await the task completion and get the number of messages processed in the batch
                batch_messages_processed = await task

                # Update total processed message count
                total_messages_processed += batch_messages_processed

                # Break the loop if no messages were processed
                if batch_messages_processed == 0:
                    consecutive_empty_batches += 1
                    if consecutive_empty_batches >= 3:  # Stop after 3 consecutive empty batches
                        print(f"No more messages to process. Total messages processed: {total_messages_processed}")
                        break
                else:
                    consecutive_empty_batches = 0  # Reset the counter if a batch was processed

            print(f"Processed {total_messages_processed} messages in total.")

# Function to start the async event loop
def start_async_message_processing(batch_size=10, max_concurrent_batches=3):
    asyncio.run(async_process_messages_continuously(batch_size=batch_size, max_concurrent_batches=max_concurrent_batches))

# Start the async process
start_async_message_processing(batch_size=10, max_concurrent_batches=5)
